环境: numpy=1.19.2; pandas=1.1.3; lightgbm=3.2.1

# 假设train.csv中数据有缺失(即假设自2017.1.1至2020.12.31每天都应该有交易数据):

## 用三种方式填充缺失值: 1. 用上一年或下一年该月的月均销量(或者年均销量)填充该年该月的缺失值; 2. 缺失值全部用0填充; 3. 用上个月和下个月的权重均值来填充这个月的缺失值(假设[20%, 80%])

### 为了防止时间浪费, 已附上缺失值处理后的csv文件, 只需读取即可

Predict Model

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import time
start_time = time.time()  # 获取当时系统时间
warnings.filterwarnings('ignore')  # 控制警告输出

path = ''  # Your path

In [ ]:
train_simple = pd.read_csv(
    path + 'train_simple.csv',
    parse_dates=['datetime'],
    index_col=['datetime'])  # train_simple.csv 是提取了 train.csv 文件里的 item_id, province_id, quantity 列
Origin = train_simple.groupby(['item_id', 'province_id', 'datetime']).sum()
Origin.to_csv('Origin.csv')

In [ ]:
origin = pd.read_csv(path + 'Origin.csv',
                     parse_dates=['datetime'],
                     index_col=['datetime'])  # 正确读取 datetime 日期列
test = pd.read_csv(
        path + 'perday.csv',  # perday.csv中的item_id, province_id 与 submission.csv相同, 不同的是 perday.csv 中包含了2021.1-2021.3的每一天
        parse_dates=['datetime'],
        index_col=['datetime'])
rawbase = pd.read_csv(
        path + 'rawbase.csv',  # rawbase.csv中的item_id, province_id 与 train.csv相同, 不同的是 rawbase.csv 中包含了2017.1-2020.12的每一天
        parse_dates=['datetime'],
        index_col=['datetime'])

In [ ]:
train = pd.merge(
        rawbase,  # 将 rawbase 和 origin 数据框合并起来, 发现缺失值
        origin,
        on=['datetime', 'item_id', 'province_id'],
        how='left')
train = train.rename(columns={'quantity_y': 'quantity'})
train = train[['item_id', 'province_id', 'quantity']]

In [ ]:
def expand(df):
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['day_of_week'] = df.index.dayofweek
    return df


data = expand(train)

used_data = data.loc[(data.year >= 2017) & (data.year<=2020)] # 此处为使用2017年及以后的数据 (同理可使用2018年及以后的数据; 2020年以前的数据)
# used_data = data.loc[(data.year >= 2017) & (data.year <= 2019)]  # 此处为使用2017年至2019年的数据

In [ ]:
# 使用 pivot_table 透视表
year_itemid_table = pd.pivot_table(used_data,
                                   index='year',
                                   columns=['item_id', 'province_id', 'month'],
                                   values='quantity',
                                   aggfunc=np.mean)
# 通过 index='year' 获取 year 信息
# 通过 values='quantity' 筛选出需要的quantity数据
# 通过 aggfunc = np.mean 计算均值
# 通过 columns='item_id' 设置列层次字段

year_itemid_fillmonth_table = pd.pivot_table(
        used_data,
        index='year',
        columns=['item_id', 'province_id'],
        values='quantity',
        aggfunc=np.mean)

### 方法一: 用上一年或下一年该月的月均销量(或者年均销量)填充该年该月的缺失值(耗时约7分钟)

In [ ]:
year_itemid_table1 = year_itemid_table.fillna(method='bfill')
year_itemid_table1 = year_itemid_table1.fillna(
    method='ffill')  # 用上一年该月的月均值或下一年该月的月均值的值填充缺失值

year_itemid_fillmonth_table1 = year_itemid_fillmonth_table.fillna(
    method='bfill')
year_itemid_fillmonth_table1 = year_itemid_fillmonth_table1.fillna(
    method='ffill')

used_data1 = used_data

In [ ]:
(indexlist, column) = np.where(np.isnan(used_data1))

for index in indexlist:
    itemid = used_data1.iloc[index].item_id  # 读取对应的 itemid, provinceid, year, month
    provinceid = used_data1.iloc[index].province_id
    year = used_data1.iloc[index].year
    month = used_data1.iloc[index].month
    if itemid not in year_itemid_table1:
        used_data1['quantity'][index] = 0  # 如果该itemid没有任何数据, 则按 0 填充缺失值
    else:
        if provinceid not in year_itemid_table1[int(itemid)]:
            used_data1['quantity'][index] = 0  # 如果该provinceid没有任何数据, 则按 0 填充缺失值
        else:
            if month in year_itemid_table1[int(itemid)][int(provinceid)]:  # 判断月份是否出现在 year_itemid_table1 里
                # 如果在就按月均填充
                used_data1['quantity'][index] = year_itemid_table1[int(itemid)][int(provinceid)][int(month)].loc[int(year)]  # 在 year_itemid_table1里进行搜索
            else:
                # 如果不在就按年均填充
                used_data1['quantity'][index] = year_itemid_fillmonth_table1[int(itemid)][int(provinceid)].loc[int(year)]

used_data1.to_csv(path + 'used_data1.csv')
# used_data1.to_csv(path + 'used_data1_ex2020.csv')    # 使用2017-2019年的数据

### 方法二: 用0填充缺失值

In [ ]:
used_data2 = used_data.fillna(0)
used_data2.to_csv(path + 'used_data2.csv')
# used_data2.to_csv(path + 'used_data2_ex2020.csv')    # 使用2017-2019年的数据

### 方法三: 用上个月和下个月的权重均值来填充这个月的缺失值(假设权重为[20%,80%])(耗时约1分钟)

In [ ]:
weight = [0.2, 0.8]
year_itemid_table3_b = year_itemid_table.fillna(method='bfill')
year_itemid_table3_b = year_itemid_table3_b.fillna(0)
year_itemid_table3_f = year_itemid_table.fillna(method='ffill')
year_itemid_table3_f = year_itemid_table3_f.fillna(0)
year_itemid_table3 = weight[0] * year_itemid_table3_f + weight[1] * year_itemid_table3_b

year_itemid_fillmonth_table3_b = year_itemid_fillmonth_table.fillna(method='bfill')
year_itemid_fillmonth_table3_b = year_itemid_fillmonth_table3_b.fillna(0)
year_itemid_fillmonth_table3_f = year_itemid_fillmonth_table.fillna(method='ffill')
year_itemid_fillmonth_table3_f = year_itemid_fillmonth_table3_f.fillna(0)
year_itemid_fillmonth_table3 = weight[0] * year_itemid_fillmonth_table3_f + weight[1] * year_itemid_fillmonth_table3_b

used_data3 = used_data

In [ ]:
(indexlist, column) = np.where(np.isnan(used_data3))

for index in indexlist:
    itemid = used_data3.iloc[index].item_id  # 读取对应的 itemid, provinceid, year, month
    provinceid = used_data3.iloc[index].province_id
    year = used_data3.iloc[index].year
    month = used_data3.iloc[index].month
    if itemid not in year_itemid_table3:
        used_data3['quantity'][index] = 0  # 如果该itemid没有任何数据, 则按 0 填充缺失值
    else:
        if provinceid not in year_itemid_table3[int(itemid)]:
            used_data3['quantity'][index] = 0  # 如果该provinceid没有任何数据, 则按 0 填充缺失值
        else:
            if month in year_itemid_table3[int(itemid)][int(provinceid)]:  # 判断月份是否出现在 year_itemid_table1 里
                # 如果在就按月均填充
                used_data3['quantity'][index] = year_itemid_table3[int(itemid)][int(provinceid)][int(month)].loc[int(year)]  # 在 year_itemid_table1里进行搜索
            else:
                # 如果不在就按年均填充
                used_data3['quantity'][index] = year_itemid_fillmonth_table3[int(itemid)][int(provinceid)].loc[int(year)]

used_data3.to_csv(path + 'used_data3.csv')
# used_data3.to_csv(path + 'used_data3_ex2020.csv')    # 使用2017-2019年的数据

### 读取生成的数据

In [ ]:
used_data = pd.read_csv(path + 'used_data1.csv', parse_dates=['datetime'], index_col=['datetime'])  # 读取方法一获取的数据
# used_data = pd.read_csv(path+'used_data2.csv', parse_dates=['datetime'], index_col=['datetime'])
# used_data = pd.read_csv(path+'used_data3.csv', parse_dates=['datetime'], index_col=['datetime'])
# used_data = pd.read_csv(path + 'used_data1_ex2020.csv', parse_dates=['datetime'], index_col=['datetime'])
# used_data = pd.read_csv(path + 'used_data2_ex2020.csv', parse_dates=['datetime'], index_col=['datetime'])
# used_data = pd.read_csv(path + 'used_data3_ex2020.csv', parse_dates=['datetime'], index_col=['datetime'])

dayofweek_itemid_table = pd.pivot_table(used_data,
                                        index='day_of_week',
                                        columns='item_id',
                                        values='quantity',
                                        aggfunc=np.mean)

quanti_avg = used_data.quantity.mean()

month_table = pd.pivot_table(
    used_data, index='month', values='quantity', aggfunc=np.mean) / quanti_avg

provinceid_table = pd.pivot_table(
    used_data, index='province_id', values='quantity', aggfunc=np.mean) / quanti_avg

year_table = pd.pivot_table(
    used_data, index='year', values='quantity', aggfunc=np.mean) / quanti_avg

years = np.arange(2017, 2022)  # 使用2017-2020全数据
# years = np.arange(2017, 2021)  # 使用2017-2019数据

In [ ]:
# 用 years 和 year_table 的 quantity 做 二次曲线拟合(同理可做三次…) 得到 annual_growth 的方程

annual_growth = np.poly1d(np.polyfit(years[:-1], year_table.values.squeeze(), 2, w=np.exp((years - 2021) / 10)[:-1]))  # 使用2017-2020全数据
# annual_growth = np.poly1d(np.polyfit(years[:-1], year_table.values.squeeze(), 2, w=np.exp((years - 2020) / 10)[:-1]))  # 使用2017-2019全数据

In [ ]:
pred_quantity = []
for index, row in test.iterrows():
    dayofweek, month, year = row.name.dayofweek, row.name.month, row.name.year  # 用 row.name 返回时间
    itemid, provinceid = row['item_id'], row['province_id']
    base_quantity = dayofweek_itemid_table.at[dayofweek, itemid]
    monthmulprovince = month_table.at[month, 'quantity'] * provinceid_table.at[provinceid, 'quantity']
    thingstoappend = np.round(base_quantity * monthmulprovince * annual_growth(year), 0)
    if thingstoappend == thingstoappend:
        pred_quantity.append(int(thingstoappend))
    else:
        pred_quantity.append(0)

In [ ]:
test['quantity'] = pred_quantity
train = train.loc[(train.index.year >= 2017) & (train.index.year <= 2020), :]  # 只使用2017及以后的数据
# train = train.loc[(train.index.year >= 2017) & (train.index.year <= 2019), :]  # 只使用2017-2019的数据
df = pd.concat([train, test], sort=False)
df.reset_index(inplace=True)

In [ ]:
def datefeatures(df):  # 对时间的特征做更深的研究
    df['month'] = df.datetime.dt.month
    df['day_of_month'] = df.datetime.dt.day
    df['day_of_year'] = df.datetime.dt.dayofyear
    df['week_of_year'] = df.datetime.dt.weekofyear
    df['day_of_week'] = df.datetime.dt.dayofweek + 1
    df['year'] = df.datetime.dt.year
    df['is_weekend'] = df.datetime.dt.weekday // 4
    df['is_month_start'] = df.datetime.dt.is_month_start.astype(int)
    df['is_month_end'] = df.datetime.dt.is_month_end.astype(int)
    df['quarter'] = df.datetime.dt.quarter
    df['week_until_now'] = [
        int(x) for x in np.floor(
            (df.datetime - pd.to_datetime('2016-12-31')).dt.days / 7) + 1
    ]
    df['quarter_until_now'] = (df['year'] - 2017) * 4 + df['quarter']
    df['week_of_month'] = df['week_of_year'].values // 4.35
    return df


df = datefeatures(df)

weekofdaylist = [
    'is_Mon', 'is_Tue', 'is_Wed', 'is_Thu', 'is_Fri', 'is_Sat', 'is_Sun'
]
for i in range(7):
    df[weekofdaylist[i]] = np.where(df['day_of_week'] == i + 1, 1, 0)

In [ ]:
featurelist = [
    'day_of_week', 'week_of_month', 'week_of_year', 'month', 'quarter',
    'is_weekend', 'day_of_week', 'week_of_month'
]
shiftlist = [0, 0, 0, 0, 0, 0, 12, 12]  # 设置偏移量数组

for feature, shift in zip(featurelist, shiftlist):
    grouped_df = df.groupby(['province_id', 'item_id', feature])['quantity'].expanding().mean().shift(shift).bfill().reset_index()
    grouped_df.columns = [
        'province_id', 'item_id', feature, 'datetime',
        feature + f'_ex_avg_quan{str(shift)}'
    ]
    grouped_df = grouped_df.sort_values(
        by=['item_id', 'province_id', 'datetime'])
    df[feature + f'_ex_avg_quan{str(shift)}'] = grouped_df[
        feature + f'_ex_avg_quan{str(shift)}'].values

In [ ]:
df.sort_values(by=['item_id', 'province_id', 'datetime'], axis=0, inplace=True)


# 生成噪音
def random_noise(df):
    return np.random.normal(scale=0.01, size=(len(df), ))


# 对不同的时间窗口生成滞后特征
def lag_features(df, laglist):
    df = df.copy()  # 深拷贝, 将函数里面的拷贝传递到函数外面
    for lag in laglist:
        df['quan_lag_' + str(lag)] = df.groupby(["item_id", "province_id"])['quantity'].transform(lambda x: x.shift(lag)) + random_noise(df)
        # 生成随机的滞后特征, 值是通过shift函数的移位加上随机噪音
    return df


laglist = [91, 98, 105, 112, 119, 126, 182, 364, 546, 728]
df = lag_features(df, laglist)


# 移动平均, 即用当前值和前2个数值取平均数, 再加上随机噪音
def roll_mean_features(df, windows):
    df = df.copy()
    for window in windows:
        df['quan_roll_mean_' + str(window)] = df.groupby(["item_id", "province_id"])['quantity'].transform(lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type="triang").mean()) + random_noise(df)
    return df


windows = [91, 182, 365, 546, 730]
df = roll_mean_features(df, windows)


# 指数加权平均特征
def ewm_features(df, alphas, lags):
    df = df.copy()
    for alpha in alphas:
        for lag in lags:
            df['quan_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
            df.groupby(["item_id", "province_id"])['quantity'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return df


alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [91, 98, 105, 112, 180, 270, 365, 546, 728]
df = ewm_features(df, alphas, lags)

In [ ]:
# One-Hot Encoding
df_dum = pd.get_dummies(df[[
        'province_id',
        'item_id',
        'day_of_week',
        'month',
        ]],
        columns=[
        'province_id',
        'item_id',
        'day_of_week',
        'month',
        ],
        dummy_na=True)
df = pd.concat([df, df_dum], axis=1)

# 转用 log
df['quantity'] = np.log1p(df["quantity"].values)

# 特征工程与数据准备完成
print(f'用时 {int(time.time()-start_time)} 秒.')
print(f'共有 {df.shape[1]} 种特征 <=---')

In [ ]:
# 模型检验(大约耗时3分钟)
start_time = time.time()

cols = [
    col for col in df.columns
    if col not in ['datetime', 'id', "quantity", "year"]
]

train = df.loc[~df.quantity.isna()]
X_train, Y_train = train[cols], train['quantity']

test = df.loc[df.id.notnull()]
X_test = test[cols]

iteration = 15000

lgb_params = {
    'nthread': -1,
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'max_depth': 7,
    'num_leaves': 28,
    'task': 'train',
    'objective': 'regression_l1',
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'lambda_l1': 0.06,
    'lambda_l2': 0.05,
    'verbose': -1,
}

# LightGBM dataset
lgbtrain_all = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)
final_model = lgb.train(lgb_params, lgbtrain_all, num_boost_round=iteration)
test_preds = final_model.predict(X_test, num_iteration=iteration)
print(f'模型计算用时 {int(time.time()-start_time)} 秒.')

In [ ]:
submission = pd.DataFrame({
    'id': [*range(len(test_preds))],
    'quantity': np.round(np.expm1(test_preds), 7)
})  # turn back to normal scale
submission.quantity[submission.quantity < 0] = 0
submission['quantity'] = submission.quantity.astype(float)
submission.to_csv('submission1.csv', index=False)
# submission.to_csv('submission2.csv', index=False)
# submission.to_csv('submission3.csv', index=False)
# submission.to_csv('submission1_ex2020.csv', index=False)
# submission.to_csv('submission2_ex2020.csv', index=False)
# submission.to_csv('submission3_ex2020.csv', index=False)
print(f'单日预测结果数据已导出.')

(1) 若考虑缺失值, 最终我们可以得到两组数据, 一组是包括疫情年的数据(即不考虑疫情影响, submission1.csv, submission2.csv, submission3.csv), 一组是不包括疫情年的数据(即考虑疫情影响, submission1_ex2020.csv, submission2_ex2020.csv, submission3_ex2020.csv).

(2)若不考虑缺失值, 我们可以得到一组数据(submission.csv)

对于组内不同方法得出的结果采用的权重为[30%, 16%, 54%], 两组间的权重为[86%, 14%], 与不考虑缺失值的结果权重为[50%, 50%]

将最终结果作为 predict_quantity 列加入到大赛要求的 submission.csv 文件中